## CHEME 5660 Lab 8: Markov Decision Process (MDP) solution of the Branched Tiger Problem

<img src="./figs/Fig-Branched-MDP-Schematic-no-a-labels.png" style="margin:auto; width:60%"/>

__Fig. 1__: Schematic of the Tiger problem modeled as an N-state, four-action (left, right, up, down) Markov decision process. The hallway has three types of paths: unobstructed (white, free), mildly obstructed (light gray, small cost), and obstructed (dark gray, large cost).

## Introduction
A Markov decision process is the tuple $\left(\mathcal{S}, \mathcal{A}, R_{a}\left(s, s^{\prime}\right), T_{a}\left(s,s^{\prime}\right), \gamma\right)$ where:

* The state space $\mathcal{S}$ is the set of all possible states $s$ that a system can exist in
* The action space $\mathcal{A}$ is the set of all possible actions $a$ that are available to the agent, where $\mathcal{A}_{s} \subseteq \mathcal{A}$ is the subset of the action space $\mathcal{A}$ that is accessible from state $s$.
* An expected immediate reward $R_{a}\left(s, s^{\prime}\right)$ is received after transitioning from state $s\rightarrow{s}^{\prime}$ due to action $a$. 
* The transition $T_{a}\left(s,s^{\prime}\right) = P(s_{t+1} = s^{\prime}~|~s_{t}=s,a_{t} = a)$ denotes the probability that action $a$ in state $s$ at time $t$ will result in state $s^{\prime}$ at time $t+1$
* The quantity $\gamma$ is a _discount factor_; the discount factor is used to weigh the _future expected utility_.

Finally, a policy function $\pi$ is the (potentially probabilistic) mapping from states $s\in\mathcal{S}$ to actions $a\in\mathcal{A}$ used by the agent to solve the decision task. 

### Value iteration
In the previous example, we saw how we could develop _a policy_ $\pi(s)$ by looking at the values in the $Q$-array. However, this required the utility vector; thus, we needed to hypothesize a policy that may not be the _optimal policy_. There are two techniques to compute optimal policies, and we'll explore the simpler of the two, namely _value iteration_. In _value iteration_, the value function (the vector of utility values) is updated iteratively using the _Bellman update__ procedure:

$$
U_{k+1}(s) = \max_{a}\left(R(s,a) + \gamma\sum_{s^{\prime}\in\mathcal{S}}T(s^{\prime} | s, a)U_{k}(s^{\prime})\right)
$$

This procedure is guaranteed to converge to the optimal utility vector (value function).  

### Problem

An agent is trapped in a long hallway with two doors at either end (Fig. 1). Behind the green door is a tiger (and certain death), while behind the red door is freedom. If the agent opens the green door, the agent is eaten (and receives a large negative reward). However, if the agent opens the red door, it escapes and gets a positive reward. 

For this problem, the MDP has the tuple components:
* $\mathcal{S} = \left\{1,2,\dots,N\right\}$ while the action set is $\mathcal{A} = \left\{a_{1},a_{2}, a_{3}, a_{4}\right\}$; action $a_{1}$ moves the agent one state to the left, action $a_{2}$ moves the agent one state to the right, action $a_{3}$ moves the agent one stop up, and action $a_{4}$ moves the agent one step down.
* The agent receives a postive reward for entering the red state $N$ (escapes). However, the agent is penalized for entering the green state $1$ (eaten by the tiger).  Finally, the agent is not charged to move to adjacent locations if those locations are unobstructed. However, there is a small charge to move through mildly obstructed locations (light gray circles) and a larger charge to move through obstructed areas (dark gray circles).
* Let the probability of correctly executing an action $a_{j}\in\mathcal{A}$ be $\alpha$.

Let's use value iteration to estimate the _optimal policy_ $\pi^{\star}(s)$

## Lab 8 setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-8-MDP-Tiger-Problem`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-8-MDP-Tiger-Problem/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-8-MDP-Tiger-Problem/Manifest.toml`


In [2]:
# load req packages -
using PrettyTables

In [3]:
include("CHEME-5660-Lab-8-CodeLib.jl");

#### Configure constants, states and actions

In [14]:
# setup some global constants -
α = 0.95; # probability of moving the direction we are expect

# setup the states and actions -
safety = 1;
tiger = 15;
left_reward = -100.0;
right_reward = 1000.0;

# by pass costs -
top_hallway_cost = -20.0;
bottom_hallway_cost = -1.0;
blocked = -10000.0;

states = range(safety,stop=tiger, step=1) |> collect;
actions = [1,2,3,4]; # a₁ = move left, a₂ = move right, a₃ = move up, a₄ = move down
γ = 0.95; # how much do we consider future moves?

# scenario and setup flags -
bottom_hallway_cave_in_flag = false;
should_run_T_check = false;

#### Configure the rewards array $R(s,a)$

In [6]:
# setup the rewards -
R = Array{Float64,2}(undef,length(states), length(actions));

# most of the rewards are zero -
fill!(R,0.0) # fill R w/zeros

# set the rewards for the ends -
R[safety + 1, 1] = left_reward; # if in state 2, and we take action 1. Bad.
R[tiger - 1, 2] = right_reward; # if in state N - 1, and we take action 2, we live, our kids are all doctors, and we are generally content

# linear nodes are blocked -
list_linear_nodes = range(10,stop=(tiger-1),step = 1) |> collect
for i ∈ list_linear_nodes
    R[i,3] = blocked;
    R[i,4] = blocked;
end

# actions:
# a₁ = left
# a₂ = right
# a₃ = up
# a₄ = down

# rewards for the by-passes. 
R[2,3] = top_hallway_cost;
R[2,4] = bottom_hallway_cost;
R[2,2] = blocked;
R[9,1] = blocked;
R[9,3] = top_hallway_cost;
R[9,4] = bottom_hallway_cost;

# top -
R[3,1] = blocked;
R[3,2] = top_hallway_cost
R[3,3] = blocked;
R[4,1] = top_hallway_cost
R[4,2] = top_hallway_cost
R[4,3] = blocked;
R[4,4] = blocked;
R[5,1] = top_hallway_cost
R[5,2] = blocked;
R[5,3] = blocked;

# bottom -
R[6,1] = blocked;
R[6,2] = bottom_hallway_cost;
R[6,4] = blocked;
R[7,1] = bottom_hallway_cost;
R[7,2] = bottom_hallway_cost;
R[7,3] = blocked;
R[7,4] = blocked;
R[8,1] = bottom_hallway_cost;
R[8,2] = blocked;
R[8,4] = blocked;

#### Configure the transition array $T_{a}(s,s^{\prime})$

In [7]:
# Setup the transitions
T = Array{Float64,3}(undef, length(states), length(states), length(actions));
fill!(T,0.0);

# We need to put values into the transition array (these are probabilities, so eah row much sum to 1)
T[safety, 1, 1:length(actions)] .= 1.0; # if we are in state 1, we stay in state 1 ∀a ∈ 𝒜
T[tiger, tiger, 1:length(actions)] .= 1.0; # if we are in state 5, we stay in state 5 

# left actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,1] = α;
    T[s,s+1,1] = (1-α);
end

# right actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,2] = (1-α);
    T[s,s+1,2] = α; 
end

# Node 2 -
T[2,:,2] .= 0.0
T[2,3,3] = α;
T[2,6,3] = (1-α);
T[2,6,4] = α;
T[2,3,4] = (1-α);
T[2,2,2] = 1.0; # node 2 can't go right

# Node 3 -
T[3,:,:] .= 0.0
T[3,3,1] = 1.0;
T[3,4,2] = α;
T[3,3,2] = (1-α);
T[3,3,3] = 1.0;
T[3,2,4] = α;
T[3,3,4] = (1-α);

# Node 4 -
T[4,4,3] = 1.0;
T[4,4,4] = 1.0;

# Node 5 -
T[5,:,:] .= 0.0;
T[5,4,1] = α;
T[5,5,1] = (1-α);
T[5,5,2] = 1.0;
T[5,5,3] = 1.0;
T[5,9,4] = α;
T[5,5,4] = (1-α);

# Node 6 -
T[6,:,:] .= 0.0;
T[6,6,1] = 1.0;
T[6,7,2] = α;
T[6,6,2] = (1-α);
T[6,2,3] = α;
T[6,6,3] = (1-α);
T[6,6,4] = 1.0;

# Node 7 -
T[7,7,3] = 1.0;
T[7,7,4] = 1.0;

# Node 8 -
T[8,:,:] .= 0.0
T[8,7,1] = α;
T[8,8,1] = (1-α);
T[8,8,2] = 1.0;
T[8,9,3] = α;
T[8,8,3] = (1-α);
T[8,8,4] = 1.0;

# Node 9 -
T[9,:,:] .= 0.0;
T[9,9,1] = 1.0;
T[9,10,2] = α;
T[9,9,2] = (1-α);
T[9,5,3] = α;
T[9,9,3] = (1-α);
T[9,8,4] = α;
T[9,9,4] = (1-α);

# Nodes 10 -> end
for s = 10:(tiger-1)
    T[s,s,3] = 1.0
    T[s,s,4] = 1.0
end

In [8]:
# re-configure setup to include bttom hallway cave in
if (bottom_hallway_cave_in_flag == true)
    
    # configure rewards -
    R[6,2] = blocked;
    R[8,1] = blocked;

    # configure T array -
    # node 6 -
    T[6,:,:] .= 0.0;
    T[6,6,1] = 1.0; # we stay in 6 if we go left
    T[6,6,2] = 1.0; # we stay in 6 if we go right
    T[6,2,3] = α; 
    T[6,6,3] = (1-α);
    T[6,6,4] = 1.0; # we stay in 6 if we go down
    
    # node 8 -
    T[8,:,:] .= 0.0;
    T[8,9,3] = α; 
    T[8,8,3] = (1 - α); 
    T[8,8,1] = 1.0; # we stay in 8 if we go left
    T[8,8,2] = 1.0; # we stay in 8 if we go right
    T[8,8,4] = 1.0; # we stay in 8 if we go down
end

In [9]:
if (should_run_T_check == true)
    # row summation check -
    T_array_check_table = Array{Any,2}(undef, length(states), length(actions)+1)

    for s ∈ 1:length(states)
        T_array_check_table[s,1] = s;
    end

    for a ∈ 1:length(actions)

        # sum the action table -
        Z = sum(T[:,:,a], dims=2)

        for s ∈ 1:length(states)
            T_array_check_table[s,a+1] = Z[s]
        end
    end

    # header -
    T_check_header = (["State s", "a₁ (left)", "a₂ (right)", "a₃ (up)", "a₄ (down)"]);

    # display -
    pretty_table(T_array_check_table; header=T_check_header)
end

### Build the MDP problem object and estimate $\pi(s)^{\star}$ 

In [10]:
mdp_problem = build(MDPProblem; 𝒮 = states, 𝒜 = actions, T = T, R = R, γ = γ);

In [11]:
U = solve(mdp_problem,1000);

In [12]:
# compute the Q array -
Q_array = Q(mdp_problem, U)

# compute the policy -
policy = π(Q_array);

In [13]:
# make a Q-table -
Q_table_data_array = Array{Any,2}(undef, length(states), length(actions)+3)

for s ∈ 1:length(states)
    
    Q_table_data_array[s,1] = s;
    
    direction = "left"
    policy_index = policy[s];
    if policy_index == 2
        direction = "right" 
    elseif policy_index == 3
         direction = "up" 
    elseif policy_index == 4
         direction = "down" 
    elseif policy_index == 0
        direction = "stop"
    end
    
    Q_table_data_array[s,2] = direction;
    Q_table_data_array[s,3] = policy_index;
    
    
    for a ∈ 1:length(actions)
        Q_table_data_array[s,a+3] = Q_array[s,a];
    end
end

# header -
Q_table_header = (["State", "Direction", "π(s)", "U(a₁) (left)", "U(a₂) (right)", "U(a₃) (up)", "U(a₄) (down)"])

# show -
pretty_table(Q_table_data_array; header = Q_table_header)

┌───────┬───────────┬──────┬──────────────┬───────────────┬────────────┬──────────────┐
│ State │ Direction │ π(s) │ U(a₁) (left) │ U(a₂) (right) │ U(a₃) (up) │ U(a₄) (down) │
├───────┼───────────┼──────┼──────────────┼───────────────┼────────────┼──────────────┤
│     1 │      stop │    0 │          0.0 │           0.0 │        0.0 │          0.0 │
│     2 │      down │    4 │     -70.1118 │       -9399.0 │    579.652 │      632.629 │
│     3 │     right │    2 │     -9402.24 │       629.226 │   -9402.24 │      600.836 │
│     4 │     right │    2 │      583.474 │       686.247 │   -9348.07 │     -9348.07 │
│     5 │      down │    4 │      634.936 │      -9288.04 │   -9288.04 │      749.428 │
│     6 │     right │    2 │     -9364.48 │       668.965 │    602.723 │     -9364.48 │
│     7 │     right │    2 │      638.338 │       707.134 │   -9328.22 │     -9328.22 │
│     8 │        up │    3 │      672.787 │      -9288.04 │    749.428 │     -9288.04 │
│     9 │     right │    2 │    

<img src="./figs/Fig-Branched-MDP-Schematic-no-a-labels.png" style="width:60%; align:left"/>

### Additional Resources
* [Chapter 7: Mykel J. Kochenderfer, Tim A. Wheeler, Kyle H. Wray "Algorithms for Decision Making", MIT Press 2022](https://algorithmsbook.com)

### Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.